# Model comparisons

In [2]:
import cv2
from face_alignment import FaceAlignment, LandmarksType
import os
import urllib.request as urlreq

from loreal_poc.dataloaders.loaders import DataLoader300W
from loreal_poc.dataloaders.wrappers import CroppedDataLoader

from loreal_poc.models.wrappers import OpenCVWrapper, FaceAlignmentWrapper
from loreal_poc.tests.performance import NMEMean
from loreal_poc.tests.base import Test, TestDiff
from loreal_poc.marks.facial_parts import FacialParts

In [3]:
dl = DataLoader300W(dir_path="300W/sample")
chosen_idx = 1
image, ground_truth_landmarks, _ = dl[chosen_idx]

In [4]:
facealignment_model = FaceAlignmentWrapper(model=FaceAlignment(LandmarksType.TWO_D, device="cpu", flip_input=False))
opencv_model = OpenCVWrapper()

loading data from : lbfmodel.yaml


In [5]:
# from dataclasses import fields
def create_report(models, dataloader, tests, facial_parts):
    results = []
    for model in models:
        # for __facial_part in fields(FacialParts):
        #  _facial_part = __facial_part.default
        for facial_part in facial_parts:
            for test_cls in tests:
                test_result = test_cls(metric=NMEMean, threshold=1).run(
                    model=model,
                    dataloader=CroppedDataLoader(dataloader, part=facial_part),
                    dataloader_ref=dl,
                    facial_part=facial_part,
                )
                results.append(test_result.to_dict())
    return results


report = create_report(
    [opencv_model, facealignment_model],
    dl,
    [TestDiff],
    [
        FacialParts.BOTTOM_HALF.value,
        FacialParts.UPPER_HALF.value,
        FacialParts.LEFT_HALF.value,
        FacialParts.RIGHT_HALF.value,
    ],
)

In [7]:
import pandas as pd

final_report = pd.DataFrame(report).groupby(["facial_part", "model_name", "metric_value"]).describe()

In [10]:
final_report

threshold                               \
                                           count mean std  min  25%  50%  75%   
facial_part model_name    metric_value                                          
bottom half FaceAlignment 0.071476           1.0  1.0 NaN  1.0  1.0  1.0  1.0   
            OpenCV        0.086974           1.0  1.0 NaN  1.0  1.0  1.0  1.0   
left half   FaceAlignment 0.059586           1.0  1.0 NaN  1.0  1.0  1.0  1.0   
            OpenCV        0.065393           1.0  1.0 NaN  1.0  1.0  1.0  1.0   
right half  FaceAlignment 0.076536           1.0  1.0 NaN  1.0  1.0  1.0  1.0   
            OpenCV        0.041429           1.0  1.0 NaN  1.0  1.0  1.0  1.0   
upper half  FaceAlignment 0.064830           1.0  1.0 NaN  1.0  1.0  1.0  1.0   
            OpenCV        0.027602           1.0  1.0 NaN  1.0  1.0  1.0  1.0   

                                             
                                        max  
facial_part model_name    metric_value       
bottom half FaceAlignment 0.071476      1.0  
            OpenCV        0.086974      1.0  
left half   FaceAlignment 0.059586      1.0  
            OpenCV        0.065393      1.0  
right half  FaceAlignment 0.076536      1.0  
            OpenCV        0.041429      1.0  
upper half  FaceAlignment 0.064830      1.0  
            OpenCV        0.027602      1.0